In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

from keras.models import Model
from keras.layers import *
from keras import layers, constraints, callbacks
from keras.preprocessing import sequence
from keras import optimizers, utils
from keras import backend as K

from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.sql import Row

import numpy as np

import tensorflow as tf

import random
import nltk

Using TensorFlow backend.


In [2]:
!pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/envs/py3env/lib/python3.5/site-packages (3.6.0)


In [3]:
import os
from pathlib import Path
fasttext_file = os.path.join(os.getcwd(), 'crawl-300d-2M-subword.bin')
if not Path(fasttext_file).exists():
  !gsutil cp gs://dsx_embeddings/crawl-300d-2M-subword.bin .



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying gs://dsx_embeddings/crawl-300d-2M-subword.bin...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][  6.7 GiB/  6.7 GiB]      0.0 B/s                                   
Operation completed over 1 objects/6.7 GiB.                                      


In [9]:
from gensim.models import FastText
from gensim.models.keyedvectors import Vocab

class FTLoader(FastText):
  def FTLoader():
    super(self, FastText).__init__()

  def _load_dict(self, file_handle, encoding='utf8'):
      """Load a previously saved dictionary from disk, stored in Facebook's native fasttext format.

      Parameters
      ----------
      file_handle : file-like object
          The opened file handle to the persisted dictionary.
      encoding : str
          Specifies the encoding.

      """
      vocab_size, nwords, nlabels = self.struct_unpack(file_handle, '@3i')
      # Vocab stored by [Dictionary::save](https://github.com/facebookresearch/fastText/blob/master/src/dictionary.cc)
      if nlabels > 0:
          raise NotImplementedError("Supervised fastText models are not supported")

      self.struct_unpack(file_handle, '@1q')  # number of tokens
      if self.new_format:
        pruneidx_size, = self.struct_unpack(file_handle, '@q')
        for i in range(vocab_size):
            word_bytes = b''
            char_byte = file_handle.read(1)
            # Read vocab word
            while char_byte != b'\x00':
                word_bytes += char_byte
                char_byte = file_handle.read(1)
            word = word_bytes.decode(encoding, errors='ignore')
            count, _ = self.struct_unpack(file_handle, '@qb')

            self.wv.vocab[word] = Vocab(index=i, count=count)
            self.wv.index2word.append(word)

        assert len(self.wv.vocab) == nwords, (
            'mismatch between final vocab size ({} words), '
            'and expected number of words ({} words)'.format(len(self.wv.vocab), nwords))
        
      if self.new_format:
        for j in range(pruneidx_size):
            self.struct_unpack(file_handle, '@2i')
 
  def load_binary_data(self, encoding='utf8'):
    """Load data from a binary file created by Facebook's native FastText.

    Parameters
    ----------
    encoding : str, optional
        Specifies the encoding.

    """

    # TODO use smart_open again when https://github.com/RaRe-Technologies/smart_open/issues/207 will be fixed
    with open(self.file_name, 'rb') as f:
        self._load_model_params(f)
        self._load_dict(f, encoding='utf-8')
        self._load_vectors(f)
            
fasttext_model = FTLoader.load_fasttext_format(fasttext_file)

In [11]:
fasttext_model.wv['superlicious']

array([-1.45213178e-03,  2.25556791e-02,  1.05936378e-02,  7.98907038e-03,
       -5.81766060e-03, -3.52887139e-02,  2.50215642e-04,  1.47205302e-02,
        5.83328307e-03, -6.10998869e-02, -9.60715953e-03,  7.69572752e-03,
        2.90394784e-03,  4.24098084e-03, -7.19094649e-03, -7.04404607e-04,
        3.31354700e-03,  1.35269472e-02,  9.16242972e-03, -4.52209875e-04,
       -3.64459828e-02, -5.46533801e-03, -5.34782605e-03, -2.42185090e-02,
        2.21637785e-02, -1.52887916e-03, -8.38000700e-03,  6.05652994e-03,
        1.34017328e-02,  2.82888766e-03, -6.00957824e-03, -8.51077773e-03,
        7.39412906e-04, -4.72686859e-03, -7.34564895e-03,  1.16540119e-02,
        4.17946698e-03,  9.33624338e-03, -1.78848172e-03,  2.17378279e-03,
        2.41085496e-02, -2.15340662e-03, -8.12694337e-03,  1.17682954e-02,
       -1.85280088e-02,  3.01055107e-02,  7.84501340e-03,  1.14235161e-02,
       -1.71955097e-02, -5.25120937e-04, -1.77040938e-02, -1.38701173e-02,
        5.55144576e-03,  